In [1]:
from pyspark.sql import SparkSession

In [2]:
from pyspark.sql.functions import *




In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("KafkaTest") \
    .config(
        "spark.jars",
        "/opt/spark/jars/spark-sql-kafka-0-10_2.13-4.0.1.jar,"
        "/opt/spark/jars/kafka-clients-3.7.0.jar,"
        "/opt/spark/jars/spark-token-provider-kafka-0-10_2.13-4.0.1.jar,"
        "/opt/spark/jars/mssql-jdbc-13.2.1.jre11.jar"
    ) \
    .getOrCreate()

print("✅ Spark with Kafka is ready!")



✅ Spark with Kafka is ready!


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *



JAR_PATH = "/opt/spark/jars/mssql-jdbc-13.2.1.jre11.jar" 


spark = SparkSession.builder \
    .appName("TrafficKafkaStreaming") \
    .config("spark.jars", JAR_PATH) \
    .getOrCreate()

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *


spark = SparkSession.builder.appName("TrafficKafkaStreaming").getOrCreate()


df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("subscribe", "traffic_topic") \
    .option("startingOffsets", "latest") \
    .load()


json_df = df.selectExpr("CAST(value AS STRING)")


schema = StructType([
    StructField("timestamp", StringType()),
    StructField("street_name", StringType()),
    StructField("vehicle_count", IntegerType()),
    StructField("vehicle_speed", DoubleType()),
    StructField("light_level", StringType()),
    StructField("weather", StringType()),
    StructField("traffic_light", StringType()),
    StructField("solar_energy_level", DoubleType()),
    StructField("lighting_demand", StringType())
])


parsed_df = json_df.select(from_json(col("value"), schema).alias("data")).select("data.*")


df_transformed = (parsed_df
    .withColumn("timestamp", to_timestamp(col("timestamp")))
    .withColumn("hour_of_day", hour(col("timestamp")))
    .withColumn("day_of_week", date_format(col("timestamp"), "EEEE"))
    .withColumn("Is_peak_hour", when((col("hour_of_day").between(7, 9)) | (col("hour_of_day").between(16, 18)), 1).otherwise(0))
    .withColumn("congestion_level", when((col("vehicle_count") >= 30) & (col("vehicle_speed") <= 20), "High")
                                    .when((col("vehicle_count") >= 15) & (col("vehicle_speed") <= 40), "Medium")
                                    .otherwise("Low"))
    .withColumn("Is_congested", when(col("congestion_level") == "High", 1).otherwise(0))
    .withColumn("lighting_demand_kW", when(col("lighting_demand") == "low", 0.5)
                                      .when(col("lighting_demand") == "medium", 2.0)
                                      .when(col("lighting_demand") == "high", 5.0))
    .withColumn("lighting_consumption_kWh", 
                when(col("lighting_demand_kW") - col("solar_energy_level") > 0,
                     col("lighting_demand_kW") - col("solar_energy_level"))
                .otherwise(0.0))
    .withColumn("energy_alert_flag", (col("lighting_consumption_kWh") > 2.0))
)


query = df_transformed.writeStream \
    .format("parquet") \
    .option("path", "/home/jovyan/work/data_lake/processed_data") \
    .option("checkpointLocation", "/home/jovyan/work/data_lake/checkpoints") \
    .outputMode("append") \
    .start()



In [ ]:
#***********************************************************Don not touch #

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *


JAR_PATH = "/opt/spark/jars/mssql-jdbc-13.2.1.jre11.jar" 


spark = SparkSession.builder \
    .appName("TrafficKafkaStreaming") \
    .config("spark.driver.extraClassPath", JAR_PATH) \
    .getOrCreate()




df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("subscribe", "traffic_topic") \
    .option("startingOffsets", "latest") \
    .load()


json_df = df.selectExpr("CAST(value AS STRING)")


schema = StructType([
    StructField("timestamp", StringType()),
    StructField("street_name", StringType()),
    StructField("vehicle_count", IntegerType()),
    StructField("vehicle_speed", DoubleType()),
    StructField("light_level", StringType()),
    StructField("weather", StringType()),
    StructField("traffic_light", StringType()),
    StructField("solar_energy_level", DoubleType()),
    StructField("lighting_demand", StringType())
])


parsed_df = json_df.select(from_json(col("value"), schema).alias("data")).select("data.*")


df_transformed = (parsed_df
    .withColumn("timestamp", to_timestamp(col("timestamp")))
    .withColumn("hour_of_day", hour(col("timestamp")))
    .withColumn("day_of_week", date_format(col("timestamp"), "EEEE"))
    .withColumn("Is_peak_hour", when((col("hour_of_day").between(7, 9)) | (col("hour_of_day").between(16, 18)), 1).otherwise(0))
    .withColumn("congestion_level", when((col("vehicle_count") >= 30) & (col("vehicle_speed") <= 20), "High")
                                    .when((col("vehicle_count") >= 15) & (col("vehicle_speed") <= 40), "Medium")
                                    .otherwise("Low"))
    .withColumn("Is_congested", when(col("congestion_level") == "High", 1).otherwise(0))
    .withColumn("lighting_demand_kW", when(col("lighting_demand") == "low", 0.5)
                                      .when(col("lighting_demand") == "medium", 2.0)
                                      .when(col("lighting_demand") == "high", 5.0))
    .withColumn("lighting_consumption_kWh", 
                when(col("lighting_demand_kW") - col("solar_energy_level") > 0,
                     col("lighting_demand_kW") - col("solar_energy_level"))
                .otherwise(0.0))
   
    .withColumn("energy_alert_flag", 
        (col("lighting_consumption_kWh") > 2.0).cast(IntegerType())
    ) 
)




SQL_SERVER_HOST = "host.docker.internal" 
SQL_SERVER_PORT = "1433"
DB_NAME = "final-project"
DB_USER = "sa"
DB_PASSWORD = "maya"


JDBC_URL = f"jdbc:sqlserver://{SQL_SERVER_HOST}:{SQL_SERVER_PORT};databaseName={DB_NAME};encrypt=true;trustServerCertificate=true"




def write_to_sql_server(df, epoch_id):
    """يقوم بتقسيم الدفعة إلى 3 جداول عادية (Flat Tables) وكتابتها."""
    print(f"--- Writing batch {epoch_id} to SQL Server (Starting Split) ---")

  
    jdbc_options = {
        "url": JDBC_URL,
        "user": DB_USER,
        "password": DB_PASSWORD,
        "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
    }

    try:
       
        sensors_df = df.select(
            "timestamp", "street_name", "vehicle_count", "vehicle_speed",
            "hour_of_day", "day_of_week", "Is_peak_hour", "congestion_level", 
            "Is_congested"
        )
        
        sensors_df.write.format("jdbc").options(
            dbtable="traffic_sensors_data", 
            **jdbc_options
        ).mode("append").save()
        print(f"  -> Wrote {sensors_df.count()} rows to traffic_sensors_data.")


        # B. جدول traffic_weather_conditions (الظروف البيئية)
        weather_df = df.select(
            "timestamp", "street_name", "light_level", "weather", "traffic_light"
        )
        
        weather_df.write.format("jdbc").options(
            dbtable="traffic_weather_conditions", 
            **jdbc_options
        ).mode("append").save()
        print(f"  -> Wrote {weather_df.count()} rows to traffic_weather_conditions.")


       
        energy_df = df.select(
            "timestamp", 
            "street_name",
            "solar_energy_level",
            "lighting_demand",
            "lighting_demand_kW",
            "lighting_consumption_kWh",
            "energy_alert_flag"
        )

        energy_df.write.format("jdbc").options(
            dbtable="traffic_energy_analysis", 
            **jdbc_options
        ).mode("append").save()
        print(f"  -> Wrote {energy_df.count()} rows to traffic_energy_analysis.")


        print(f"Batch {epoch_id} successfully processed and written to 3 tables.")

    except Exception as e:
        print(f"Error writing batch {epoch_id} to SQL Server: {e}")




query = df_transformed.writeStream \
    .outputMode("update") \
    .option("checkpointLocation", "/home/jovyan/work/data_lake/checkpoints_sql_split") \
    .foreachBatch(write_to_sql_server) \
    .start()

query.awaitTermination()

--- Writing batch 187 to SQL Server (Starting Split) ---
  -> Wrote 1503 rows to traffic_sensors_data.
  -> Wrote 1503 rows to traffic_weather_conditions.
  -> Wrote 1503 rows to traffic_energy_analysis.
Batch 187 successfully processed and written to 3 tables.
--- Writing batch 188 to SQL Server (Starting Split) ---
  -> Wrote 5 rows to traffic_sensors_data.
  -> Wrote 5 rows to traffic_weather_conditions.
  -> Wrote 5 rows to traffic_energy_analysis.
Batch 188 successfully processed and written to 3 tables.
--- Writing batch 189 to SQL Server (Starting Split) ---
  -> Wrote 1 rows to traffic_sensors_data.
  -> Wrote 1 rows to traffic_weather_conditions.
  -> Wrote 1 rows to traffic_energy_analysis.
Batch 189 successfully processed and written to 3 tables.
--- Writing batch 190 to SQL Server (Starting Split) ---
  -> Wrote 1 rows to traffic_sensors_data.
  -> Wrote 1 rows to traffic_weather_conditions.
  -> Wrote 1 rows to traffic_energy_analysis.
Batch 190 successfully processed and 

In [ ]:
!rm -rf work